## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**
* I/O 禁止、`print` / `sort_values` 使用禁止
* 判定は `product_id` 基準、出力列・順序は `['product_id', 'product_name']`

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  2019-01-01〜2019-03-31（2019 年 Q1）の期間にのみ販売された商品を求める。
  具体的には、各 `product_id` について:

  * Q1 期間に 1 回以上販売されている
  * かつ Q1 以外の期間では 1 回も販売されていない

  ものを抽出する。

* 入力 DF: `{{INPUT_DATAFRAMES}}`

  ```text
  Product  : 列 ['product_id', 'product_name', 'unit_price']
  Sales    : 列 ['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']
  ```

* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`

  * 列: `['product_id', 'product_name']`
  * 行: 上記条件を満たす `product_id` のみ

---

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → `groupby` で min/max 日付集約 → 条件抽出 → `isin` でセミジョイン、という素直なパターンで実装します。

```python
import pandas as pd

def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['product_id', 'product_name']
    """
    # 1) 列最小化（この問題で必要なのは product_id と sale_date のみ）
    s = sales[['product_id', 'sale_date']]

    # 2) product_id ごとに最小日付・最大日付を集約
    agg = (
        s.groupby('product_id', as_index=False)['sale_date']
          .agg(min_date='min', max_date='max')
    )

    # 3) 「すべての販売日が 2019Q1 に収まっている」product_id を抽出
    q1_start = pd.Timestamp('2019-01-01')
    q1_end = pd.Timestamp('2019-03-31')

    mask = (agg['min_date'] >= q1_start) & (agg['max_date'] <= q1_end)
    q1_only_ids = agg.loc[mask, 'product_id']

    # 4) Product から該当 product_id のみセミジョイン
    out = product.loc[product['product_id'].isin(q1_only_ids), ['product_id', 'product_name']]

    return out

Analyze Complexity
Runtime 351 ms
Beats 79.45%
Memory 69.55 MB
Beats 81.99%

```

* `sort_values` は一切使用せず、`groupby` 集約と `isin` のみで完結させています。
* 出力列の順序も指定どおり `['product_id', 'product_name']` に固定しています。

---

## 3) アルゴリズム説明

使用 API と流れを整理します。

1. **列最小化**:

   ```python
   s = sales[['product_id', 'sale_date']]
   ```

   * 後続処理に不要な列（seller_id, buyer_id, quantity, price）はここで捨ててメモリと処理コストを削減。

2. **グループ処理（最小/最大日付の集約）**:

   ```python
   agg = (
       s.groupby('product_id', as_index=False)['sale_date']
         .agg(min_date='min', max_date='max')
   )
   ```

   * 各 `product_id` について

     * `min_date`: その商品の最も古い販売日
     * `max_date`: その商品の最も新しい販売日
   * 「すべての販売日が Q1 にある」ことは
     `min_date >= '2019-01-01'` かつ `max_date <= '2019-03-31'`
     と同値になるので、`BOOL_OR` などのフラグ集計よりもシンプルです。

3. **条件抽出**:

   ```python
   q1_start = pd.Timestamp('2019-01-01')
   q1_end = pd.Timestamp('2019-03-31')

   mask = (agg['min_date'] >= q1_start) & (agg['max_date'] <= q1_end)
   q1_only_ids = agg.loc[mask, 'product_id']
   ```

   * `mask` で「Q1 だけで売れている」product_id を絞り込み、その ID シリーズを取得。

4. **軽量セミジョイン (`isin`)**:

   ```python
   out = product.loc[product['product_id'].isin(q1_only_ids),
                     ['product_id', 'product_name']]
   ```

   * 単一キー → 行のフィルタには `merge` よりも `isin` が軽くて読みやすいパターン。
   * ここで Product 側から必要な列だけを投影して最終結果にしている。

---

### NULL / 重複 / 型の扱い

* `sale_date` が NULL の行があった場合

  * `min` / `max` はデフォルトで非 NULL の値だけを対象にするため、NULL の存在で壊れにくい設計。
  * 全て NULL の場合は `min_date`, `max_date` が NULL となり、Q1 条件に該当しなくなる（妥当）。

* 重複行（同じ `product_id`・`sale_date` の行が複数）

  * `groupby` 集約では重複があっても `min_date` / `max_date` の値は変わらないため、追加コストは O(重複数) だけでロジックは影響なし。

* 型

  * LeetCode 想定では `sale_date` はすでに `datetime64[ns]` 相当の型で渡される想定。
  * 明示的に `pd.Timestamp` を使って比較することで、文字列比較ではなく日時比較で安全に評価している。

---

## 4) 計算量（概算）

`N = Sales の行数`, `P = Product の行数`, `G = 異なる product_id の個数` とします。

1. `groupby('product_id').agg(min, max)`

   * ハッシュグループ化想定で **O(N)**〜**O(N log G)** 近辺
   * `sale_date` の min/max 計算は 1 行あたり O(1)

2. `isin` によるセミジョイン

   * `q1_only_ids` の長さを `K` とすると

     * ハッシュセット化で **O(K)**
     * Product 側の走査で **O(P)**
   * 合計は **O(P + K)** 程度（`K <= G <= N`）

トータルで **O(N + P)** 近似の線形オーダーに収まり、
`sort_values` や複雑な rank を使わない分、実装もパフォーマンスもバランスが良い構成です。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Product DF]
  B[入力 Sales DF]
  C[Sales から product_id と sale_date のみ抽出]
  D[product_id ごとに min と max の sale_date を集約]
  E[集約結果から 2019Q1 だけで売れている product_id を抽出]
  F[Product DF から該当 product_id のみフィルタ]
  G[出力 product_id と product_name]
  B --> C
  C --> D
  D --> E
  A --> F
  E --> F
  F --> G
```

この実装は、PostgreSQL 版で行った「min/max 日付で Q1 に収まるか判定」という戦略を、そのまま Pandas 向けに最適化した形になっています。

結論から言うと、そのスコアならかなり仕上がっています。
アルゴリズム自体はほぼ最適クラスなので、残りは「微妙なオーバーヘッドを減らすかどうか」の世界です。

それでもあえていじるとしたら、このくらいです。

---

## 1) ほぼ最終形のコード例

* 中間の `s` をやめて `groupby` を直にかける
* マスク計算時に `Series` ではなく `ndarray` を使って、インデックスアラインのオーバーヘッドを削る

```python
import pandas as pd

def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['product_id', 'product_name']
    """
    # 1) product_id ごとに最小日付・最大日付を集約（列も最小限）
    agg = (
        sales.groupby('product_id', as_index=False)['sale_date']
             .agg(min_date='min', max_date='max')
    )

    # 2) 「すべての販売日が 2019Q1 に収まっている」product_id を抽出
    q1_start = pd.Timestamp('2019-01-01')
    q1_end = pd.Timestamp('2019-03-31')

    min_date = agg['min_date'].values
    max_date = agg['max_date'].values
    mask = (min_date >= q1_start) & (max_date <= q1_end)

    q1_only_ids = agg.loc[mask, 'product_id']

    # 3) Product から該当 product_id のみセミジョイン
    out = product.loc[
        product['product_id'].isin(q1_only_ids),
        ['product_id', 'product_name']
    ]

    return out

Analyze Complexity
Runtime 331 ms
Beats 94.28%
Memory 69.50 MB
Beats 81.99%

```

### 微調整ポイント

* `sales[['product_id', 'sale_date']]` という別変数を作らず、そのまま groupby している
  → ほんの少しだけ一時オブジェクトが減ります（効果はごく小さいですが無駄はない）。
* `mask` 計算を `Series` 同士ではなく `ndarray` 同士にしている
  → インデックス合わせなどの内部処理がいらなくなる分、わずかに軽くなり得ます。

---

## 2) これ以上やるなら「可読性とのトレードオフ」

アルゴリズムは

* 1 回の `groupby` で min/max を取る
* `isin` で Product とセミジョイン

という構造で、計算量的にも実質 O(N) に近い形です。

これ以上の改善は

* dtype を事前に `datetime64[ns]` に統一しておく
* 変数名・ローカル変数の数をさらに削る

といったレベルになり、可読性とトレードオフになる割に LeetCode の ms 単位ではほぼ誤差です。

今の 351 ms / 79%・メモリ 69.55MB / 81% なら、
「アルゴリズムも実装も合格点、余力があれば micro-tuning で遊べる状態」と見てよいと思います。

